In [ ]:
import logging
from noisepy.seis.io.compositerawstore import CompositeRawStore
from noisepy.seis.io.scedc_s3store import SCEDCS3DataStore, NCEDCS3DataStore  
from noisepy.seis.io.channelcatalog import XMLStationChannelCatalog
from datetimerange import DateTimeRange
from datetime import datetime, timezone  
from noisepy.seis.io.channel_filter_store import channel_filter

logger = logging.getLogger("noisepy.seis.io")
logger.setLevel("DEBUG")
logger.addHandler(logging.StreamHandler())

In [ ]:
SCEDC_DATA = "s3://scedc-pds/continuous_waveforms/"
NCEDC_DATA = "s3://ncedc-pds/continuous_waveforms/NC/"
S3_STORAGE_OPTIONS = {"s3": {"anon": True}}
SCEDC_STATION_XML = "s3://scedc-pds/FDSNstationXML/CI/"  
NCEDC_STATION_XML = "s3://ncedc-pds/FDSNstationXML/NC/"

# timeframe for analysis
start = datetime(2012, 1, 1, tzinfo=timezone.utc)
end = datetime(2012, 1, 3, tzinfo=timezone.utc)
timerange = DateTimeRange(start, end)
scedc_stations = "RPV,SVD,BBR".split(",")
ncedc_stations = "KCT,KRP,KHMB".split(",") 


scedc_catalog = XMLStationChannelCatalog(SCEDC_STATION_XML, storage_options=S3_STORAGE_OPTIONS) # Station catalog
ncedc_catalog = XMLStationChannelCatalog(NCEDC_STATION_XML, storage_options=S3_STORAGE_OPTIONS) # Station catalog

scedc_store = SCEDCS3DataStore(SCEDC_DATA, scedc_catalog,  channel_filter(["CI"], scedc_stations, ["*"]), timerange, storage_options=S3_STORAGE_OPTIONS) # Store for reading raw data from S3 bucket
ncedc_store = NCEDCS3DataStore(NCEDC_DATA, ncedc_catalog, channel_filter(["BK"], ncedc_stations, ["*"]), timerange, storage_options=S3_STORAGE_OPTIONS) # Store for reading raw data from S3 bucket

store = CompositeRawStore({"CI": scedc_store, "BK": ncedc_store}) # Composite store for reading data from both SCEDC and NCEDC

In [ ]:
ts = store.get_timespans()
print(ts)
channels = store.get_channels(ts[0])
print(channels)
data = store.read_data(timerange, channels[0])
print(data.data.shape)
